In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### 1. 문서 로드

In [3]:
file_path = "../0_data/Sustainability_report_2024_kr.pdf"

print("문서로드중")
loader = PyPDFLoader(file_path)
docs = loader.load()

문서로드중


In [4]:
len(docs)

83

In [5]:
docs[:10][1]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../0_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 1, 'page_label': '2'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024\nCEO 메시지\n회사 소개\n이해관계자 소통\nOur Company\n04\n05\n06\n준법과 윤리경영\nPrinciple\n53\n중대성 평가\nMateriality Assessment\n08\n임직원\n공급망\n사회공헌\n개인정보보호/보안\n고객의 안전/품질\nPeople\n31\n39\n45\n48\n50\n경제성과\n사회성과\n환경성과\n지역별 수자원 현황   \n사업부문별 환경성과\nFacts & Figures\n56\n57\n62\n65\n66\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서\nScope 3 온실가스 배출량 검증 의견서\nGRI Index\nTCFD 대조표\nSASB 대조표\n전사차원의 기후변화 대응 협력 활동\nAbout This Report\nAppendix\n70\n71\n72\n74\n77\n79\n81\n82\n[DX부문] \n추진체계 및 주요성과\n기후변화\n자원순환\n수자원 및 오염물질\n[DS부문]  \n추진체계 및 주요성과 \n기후변화\n수자원\n폐기물\n오염물질\nPlanet\n12\n13\n15\n17\n19\n20\n23\n26\n28\n삼성전자 지속가능경영보고서 2024 02Our Co

### 2. 텍스트 스플리터(청킹)

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(len(chunks), "전체 잘린 chunk 사이즈")

207 전체 잘린 chunk 사이즈


In [7]:
print(chunks[10].page_content)

· 업종 간 협력
·  기후 대응을 포함한  
UN SDGs에 대한 기여
·  투명하고 신속한 정보 공개
·  기업 간담회
·  NGO 미팅
·  이해관계자 포럼
·  시민사회 - 경영진 간담회
·  노동인권 이해관계자 워크샵
·  지속가능경영 웹사이트
· 산업협회
·  글로벌 NGO 대상 의견 수렴
·  RBA
1)
, RMI
2)
, BSR
3)
 활동
1) Responsible Business Alliance
2) Responsible Minerals Initiative
3) Business for Social Responsibility
·  UNGC
1)
·  ACEC
2)
, SCC
3)
 활동
1) United Nations Global Compact
2) Asia Clean Energy Coalition
3) Semiconductor Climate Consortium
정부 ·  간접 경제효과(투자, 고용 등 파생효과)
·  공정거래
·  안전·보건
·  컴플라이언스
·  기업윤리
·  정책 간담회
·  국회
·  정책수립 공청회
·  정책자문기구
·  지속가능경영 웹사이트
·  정부와 협업하여 중소기업 지원 프로그램 운영 ·  정부와 협업하여 벤처투자 창구 설립·운영
언론 ·  주요 제품/사업 실적 및 전략
·  투자, R&D, M&A,  
신사업 등 미래 성장 전략 
·  탄소중립 등 ESG 추진 성과
·  인/노사, 환경안전, 특허,  
제품·서비스 품질 등
·  보도자료
·  지속가능경영 웹사이트
·  삼성전자 반도체 뉴스룸
·  삼성전자 뉴스룸
· 미디어 간담회
·  글로벌 IT 전시회·신제품 발표 취재지원
·  미디어데이 개최
·  기자회견
·  인터뷰
·  기획홍보
 Our Company


### 3. 임베딩 생성 chromadb 저장

In [8]:
db_path = "../7_vectorstore/chromadb_rag_basic"

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_path,
    collection_name="samsung_2024"
)
print("벡터 저장소 저장 완료")

벡터 저장소 저장 완료


### 4. 검색기 구성(retriever)

In [9]:
retriever = vectorstore.as_retriever()
result = retriever.invoke("이 보고서에서 2024년 주요 지속 가능 경영 목표가 무엇인가요?")
result

[Document(metadata={'page': 3, 'producer': 'Adobe PDF Library 15.0', 'trapped': '/False', 'page_label': '4', 'total_pages': 83, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2024-11-25T11:10:46+09:00', 'source': '../0_data/Sustainability_report_2024_kr.pdf', 'creationdate': '2024-11-25T11:10:32+09:00'}, page_content='삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능

In [10]:
for item in result:
    print(item.page_content[:50])
    print("-"*50)

삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateria
--------------------------------------------------
삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateria
--------------------------------------------------
A Journey Towards  
a Sustainable Future
삼성전자 지속가능
--------------------------------------------------
A Journey Towards  
a Sustainable Future
삼성전자 지속가능
--------------------------------------------------


### 5. 기본 체인 만들기

In [26]:
# 1. 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라
     컨텍스트에 없으면 "문서에 근거 없음"이라고 말해라
     
     [컨텍스트]
     {context}
     """),
     ("human", "{question}")
])

# 2. 모델 선택
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

# 3. outputparser 선택
outputparser = StrOutputParser()

# 4. chain 생성
chain = rag_prompt | model | outputparser

### 6. RAG 체인 만들기

In [19]:
# 문서 합치는 함수
def format_docs(docs):
    tmp_docs = []
    for item in docs:
        tmp_docs.append(item.page_content)
    return "\n\n---\n\n".join(tmp_docs)

In [29]:
# rag_chain 만들기
rag_chain = (
    {"context" : RunnableLambda(lambda x: x) | retriever | RunnableLambda(format_docs),
    "question" : RunnablePassthrough()
    }
    | chain
)
rag_chain

{
  context: RunnableLambda(lambda x: x)
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000021C58F89790>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라\n     컨텍스트에 없으면 "문서에 근거 없음"이라고 말해라\n\n     [컨텍스트]\n     {context}\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021C601A35D0>, async_client=<openai.resources.chat.completions.comple

In [30]:
result = rag_chain.invoke(
    "2024년의 주요 목표는 무엇인가요?"
)
print(result)

2024년 주요 목표는 다음과 같습니다:
- DS부문 탄소중립(Scope 1, 2) 달성 목표
- 모든 사업장 대상 폐기물 매립 제로 인증 최고 등급(플래티넘) 획득 목표
- DS부문 국내 사업장 폐기물 재활용률 99.9% 달성 목표
- DS 영향권 생태 현황도 구축 범위 확대 추진(평택, 천안, 온양)
- DS부문 국내 사업장 취수량 증가 제로화
- 근로자가 화학물질에 직접 노출되는 구간 검출률 제로화
- 대기·수질오염물질 고효율 처리시설 및 오염물질 미사용 공정 개발
